In [2]:
import sqlite3, pandas as pd, os, re

In [6]:
csvs = [file for file in os.listdir('Merged_Data') if file.endswith('.csv')]
csvs += ['Congress_2.csv']
num = re.compile(r'(?<=_)(\d{1,2}[.csv])')
ns = []
for c in csvs:
    if len(re.findall(num, c)) >0:
        ns += re.findall(num, c)
    else:
        ns += ['0.']
# open highest number
ns = [int(i[:-1]) for i in ns]

file = csvs[ns.index(max(ns))]
data = pd.read_csv('Merged_Data/' + file)
print(data['CONGRESS'].value_counts())
data.head()

CONGRESS
5    3800
4    2536
3    2307
2    1611
1     544
Name: count, dtype: int64


,NAME,VOTES,COMMITTEE,CMTE_TYPE,MONTH,DAY,YEAR,CONGRESS,PAGE,id,...,unaccentedFamilyName,birthYear,deathYear,congresses,congressNumber,position,state,parties,congressYear,age
0,bassett,1,Cmte on Consuls,NaN,Dec,16,1790,1,3,B000226,...,bassett,1745.0,1815.0,"[{""position"":""Senator"",""congressNumber"":1,""sta...",1.0,Senator,DE,Anti-Administration,1789.0,44.0
1,butler,2,Cmte on Consuls,NaN,Dec,16,1790,1,3,B001186,...,butler,1744.0,1822.0,"[{""position"":""Delegate"",""congressNumber"":1,""st...",1.0,Senator,SC,Pro-Administration,1789.0,45.0
2,dalton,1,Cmte on Consuls,NaN,Dec,16,1790,1,3,D000013,...,dalton,1738.0,1817.0,"[{""position"":""Senator"",""congressNumber"":1,""sta...",1.0,Senator,MA,Pro-Administration,1789.0,51.0
3,dickinson,1,Cmte on Consuls,NaN,Dec,16,1790,1,3,D000324,...,dickinson,1739.0,1809.0,"[{""position"":""Delegate"",""congressNumber"":1,""st...",1.0,Senator,NJ,Pro-Administration,1789.0,50.0
4,ellsworth,12,Cmte on Consuls,NaN,Dec,16,1790,1,3,E000147,...,ellsworth,1745.0,1807.0,"[{""position"":""Delegate"",""congressNumber"":2,""st...",1.0,Senator,CT,Pro-Administration,1789.0,44.0


In [7]:
data.columns = [c.lower() for c in data.columns]
data.columns

Index(['name', 'votes', 'committee', 'cmte_type', 'month', 'day', 'year',
       'congress', 'page', 'id', 'givenname', 'familyname',
       'unaccentedfamilyname', 'birthyear', 'deathyear', 'congresses',
       'congressnumber', 'position', 'state', 'parties', 'congressyear',
       'age'],
      dtype='object')

## tSenator

In [8]:
sens = data[['id', 'givenname', 'unaccentedfamilyname', 'congress', 'age', 'position', 'state', 'parties']]
sens.head()

,id,givenname,unaccentedfamilyname,congress,age,position,state,parties
0,B000226,Richard,bassett,1,44.0,Senator,DE,Anti-Administration
1,B001186,Pierce,butler,1,45.0,Senator,SC,Pro-Administration
2,D000013,Tristram,dalton,1,51.0,Senator,MA,Pro-Administration
3,D000324,Philemon,dickinson,1,50.0,Senator,NJ,Pro-Administration
4,E000147,Oliver,ellsworth,1,44.0,Senator,CT,Pro-Administration


In [9]:
sens['age'] = sens['age'].astype('Int32')

C:\Users\elswl\AppData\Local\Temp\ipykernel_14448\785223668.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sens['age'] = sens['age'].astype('Int32')


In [10]:
sens = sens.rename(columns={'givenname' : 'first_name', 'unaccentedfamilyname' : 'last_name',
                     'parties' : 'party'})

In [11]:
sens['id'].value_counts()

id
L000067    432
L000364    425
B000929    380
M000166    323
R000448    316
          ... 
D000013     16
W000114     16
H000661     14
E000155      7
S000524      2
Name: count, Length: 79, dtype: int64

In [12]:
tSenator = sens.drop_duplicates(subset=['id', 'congress'])
tSenator['id'].value_counts()

id
L000067    5
F000313    5
H000508    5
G000526    5
K000212    4
          ..
W000202    1
D000069    1
W000287    1
P000354    1
J000005    1
Name: count, Length: 79, dtype: int64

In [13]:
tSenator

,id,first_name,last_name,congress,age,position,state,party
0,B000226,Richard,bassett,1,44,Senator,DE,Anti-Administration
1,B001186,Pierce,butler,1,45,Senator,SC,Pro-Administration
2,D000013,Tristram,dalton,1,51,Senator,MA,Pro-Administration
3,D000324,Philemon,dickinson,1,50,Senator,NJ,Pro-Administration
4,E000147,Oliver,ellsworth,1,44,Senator,CT,Pro-Administration
...,...,...,...,...,...,...,...,...
8185,W000202,James,watson,5,47,Senator,NY,Federalist
8262,D000069,Franklin,davenport,5,42,Senator,NJ,Federalist
8574,W000287,William,wells,5,28,Senator,DE,Federalist
8850,P000354,Charles,pinckney,5,40,Senator,SC,Democratic Republican


## tCommittee

In [15]:
cmtes = data[['committee','month', 'day', 'year', 'congress', 'page', 'cmte_type']]
cmtes = cmtes.rename(columns = {'cmte_type':'committee_type'})
cmtes_id = cmtes.drop_duplicates(subset = ['committee', 'day', 'year'])
cmtes_id['committee_id'] = range(len(cmtes_id))
cmtes_id

C:\Users\elswl\AppData\Local\Temp\ipykernel_14448\938507587.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cmtes_id['committee_id'] = range(len(cmtes_id))


,committee,month,day,year,congress,page,committee_type,committee_id
0,Cmte on Consuls,Dec,16,1790,1,3,NaN,0
16,cmte on Invalid bill,n.d.,n.d.,n.d.,1,3,NaN,1
32,cmte on foreign officers,Feb,12,1791,1,3,NaN,2
53,cmte on the military,Feb,14,1791,1,3,NaN,3
76,Cmte on Donald Campbell petition,Feb,15,1791,1,3,NaN,4
...,...,...,...,...,...,...,...,...
10715,Cmte bill duties on foreign and domestic spirits,Mar,23,1798,5,9,NaN,651
10734,Cmte on Bingham's motion to amend bill naval a...,Apr,3,1798,5,9,NaN,652
10756,Cmte of Conference,Apr,9,1798,5,9,NaN,653
10768,Cmte recommit bill City of Washington,Apr,10,1798,5,9,NaN,654


In [21]:
tCommittee = cmtes_id[['committee_id', 'committee', 'committee_type','month', 'day', 'year', 'congress', 'page']]
tCommittee.head()

,committee_id,committee,committee_type,month,day,year,congress,page
0,0,Cmte on Consuls,NaN,Dec,16,1790,1,3
16,1,cmte on Invalid bill,NaN,n.d.,n.d.,n.d.,1,3
32,2,cmte on foreign officers,NaN,Feb,12,1791,1,3
53,3,cmte on the military,NaN,Feb,14,1791,1,3
76,4,Cmte on Donald Campbell petition,NaN,Feb,15,1791,1,3


## tVotes

In [17]:
votes = data[['id', 'votes', 'committee', 'day', 'year']]
votes = votes.merge(tCommittee, on = ['committee', 'day', 'year'], how = 'left')
votes = votes[['id', 'votes', 'committee_id']]
votes.head()


,id,votes,committee_id
0,B000226,1,0
1,B001186,2,0
2,D000013,1,0
3,D000324,1,0
4,E000147,12,0


In [18]:
tVotes = votes

In [34]:
sql = "INSERT INTO " + 'tVotes' + " (" + \
           ",".join([c for c in tVotes.columns]) + \
           ") VALUES (" + ",".join([':' + c for c in tVotes.columns]) + ");"
sql

'INSERT INTO tVotes (id,votes,committee_id) VALUES (:id,:votes,:committee_id);'

## Database

In [25]:
conn = sqlite3.connect('data.db')
curs = conn.cursor()
curs.execute("PRAGMA foreign_keys=ON;")
pd.read_sql("PRAGMA foreign_keys;", conn)

,foreign_keys
0,1


In [26]:
print(tSenator.columns)
print(tSenator.head())

Index(['id', 'first_name', 'last_name', 'congress', 'age', 'position', 'state',
       'party'],
      dtype='object')
        id first_name  last_name  congress  age position state  \
0  B000226    Richard    bassett         1   44  Senator    DE   
1  B001186     Pierce     butler         1   45  Senator    SC   
2  D000013   Tristram     dalton         1   51  Senator    MA   
3  D000324   Philemon  dickinson         1   50  Senator    NJ   
4  E000147     Oliver  ellsworth         1   44  Senator    CT   

                 party  
0  Anti-Administration  
1   Pro-Administration  
2   Pro-Administration  
3   Pro-Administration  
4   Pro-Administration  


In [27]:
curs.execute('DROP TABLE IF EXISTS tSenator')
curs.execute("""CREATE TABLE tSenator(
                id TEXT PRIMARY KEY,
                first_name TEXT NOT NULL,
                last_name TEXT NOT NULL,
                congress INTEGER NOT NULL CHECK (congress < 17),
                age INTEGER,
                postion TEXT NOT NULL, 
                state TEXT NOT NULL CHECK (length(state) == 2),
                party TEXT);""")

In [28]:
print(tCommittee.columns)
print(tCommittee.head())

Index(['committee_id', 'committee', 'committee_type', 'month', 'day', 'year',
       'congress', 'page'],
      dtype='object')
    committee_id                         committee  committee_type month  \
0              0                   Cmte on Consuls             NaN   Dec   
16             1              cmte on Invalid bill             NaN  n.d.   
32             2          cmte on foreign officers             NaN   Feb   
53             3              cmte on the military             NaN   Feb   
76             4  Cmte on Donald Campbell petition             NaN   Feb   

     day  year  congress page  
0     16  1790         1    3  
16  n.d.  n.d.         1    3  
32    12  1791         1    3  
53    14  1791         1    3  
76    15  1791         1    3  


In [30]:
curs.execute('DROP TABLE IF EXISTS tCommittee')
curs.execute("""CREATE TABLE tCommittee(
                committee_id INTEGER PRIMARY KEY,
                committee TEXT NOT NULL,
                committee_type TEXT,
                month TEXT NOT NULL CHECK (month LIKE '___' OR 'n.d.'),
                day TEXT NOT NULL CHECK (day LIKE '__' OR 'n.d.'),
                year TEXT NOT NULL CHECK (length(year) == 4),
                congress INTEGER NOT NULL CHECK (congress < 17),
                page TEXT NOT NULL CHECK ((length(page) < 3) OR (length(page) == 5)));""")

In [32]:
print(tVotes.columns)
tVotes.head()

Index(['id', 'votes', 'committee_id'], dtype='object')


,id,votes,committee_id
0,B000226,1,0
1,B001186,2,0
2,D000013,1,0
3,D000324,1,0
4,E000147,12,0


In [33]:
curs.execute('DROP TABLE IF EXISTS tVotes')
curs.execute("""CREATE TABLE tVotes(
                id TEXT REFERENCES tSenator(id),
                votes INTEGER NOT NULL CHECK (votes < 100),
                committee_id INTEGER REFERENCES tCommittee(committee_id));""")